In [5]:
import os.path
import sys

import bqplot
import bqplot.pyplot
import ipywidgets
from IPython.display import display, HTML
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

%matplotlib ipympl
plt.style.use("ggplot")

from solution import landfillmethane
from solution import solarpvroof
from solution import solarpvutil
import ui.dd_nb as dd

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

options = ['solarpvutil:' + str(scenario) for scenario in solarpvutil.scenarios.keys()]
options += ['solarpvroof:' + str(scenario) for scenario in solarpvroof.scenarios.keys()]
options += ['landfillmethane:' + str(scenario) for scenario in landfillmethane.scenarios.keys()]
solution_choices = ipywidgets.SelectMultiple(
    options=sorted(options, key=dd.scenario_sort_key), description='Scenarios',
    disabled=False, rows=8, layout=ipywidgets.Layout(width='500px')
)
solution_choices

SelectMultiple(description='Scenarios', layout=Layout(width='500px'), options=('landfillmethane:PDS-0p2050-Pla…

In [6]:
#%%prun -s cumulative -l 200
if len(solution_choices.value):
    solutions = []
    for val in solution_choices.value:
        soln, scenario = val.split(':')
        if soln == 'solarpvutil':
            solutions.append(solarpvutil.SolarPVUtil(scenario=scenario))
        elif soln == 'solarpvroof':
            solutions.append(solarpvroof.SolarPVRoof(scenario=scenario))
        elif soln == 'landfillmethane':
            solutions.append(landfillmethane.LandfillMethane(scenario=scenario))
else:
    solutions = [solarpvutil.SolarPVUtil()]

# ------------------ Detailed Results -----------------

def blue_label(text):
    o = ipywidgets.Output()
    div = '<div style="background-color:LightSkyBlue;border:1px solid black;font-weight:bold;text-align:center;font-size:0.875em;">'
    with o:
        display(HTML(div + text + '</div>'))
    return o

def fullname(s):
    return type(s).__name__ + ':' + s.scenario


key_results_label = blue_label('The Key Adoption Results')

# --- Unit Adoption ---

unit_adoption_text = []
for s in solutions:
    pds_vs_ref = s.ua.soln_pds_tot_iunits_reqd().loc[2050, 'World'] - s.ua.soln_ref_tot_iunits_reqd().loc[2050, 'World']
    unit_adoption_text.append([fullname(s), pds_vs_ref, s.ua.soln_net_annual_funits_adopted().loc[2050, 'World'] ])
unit_adoption_text.append(['', 'TW', 'TWh'])
unit_adoption_columns = ['Scenario', 'Implementation Unit Adoption Increase in 2050 (PDS vs REF)',
                         'Functional Unit Adoption Increase in 2050 (PDS vs REF)']
unit_adoption_heading = ipywidgets.Output()
with unit_adoption_heading:
    display(HTML(pd.DataFrame(unit_adoption_text, columns=unit_adoption_columns).to_html(index=False, float_format=lambda x: '%.3f' % x)))

# --- Adoption ---

adoption_text = []
for s in solutions:
    adoption_text.append([fullname(s),
                          s.ua.soln_pds_tot_iunits_reqd().loc[2050, 'World'],
                          s.ht.soln_pds_funits_adopted().loc[2050, 'World'],
                          '', '', ''])
adoption_text.append(['', 'TW', 'TWh', '%', '%', '%'])
adoption_columns = ['Scenario', 'Global Solution Implementation Units in 2050',
                    'Global Solution Functional Units in 2050', 'Global Solution Adoption Share - 2014',
                    'Global Solution Adoption Share - 2020', 'Global Solution Adoption Share - 2050']
adoption_heading = ipywidgets.Output()
with adoption_heading:
    display(HTML(pd.DataFrame(adoption_text, columns=adoption_columns).to_html(index=False, float_format=lambda x: '%.2f' % x)))

adoption_graphs = ipywidgets.Output()
with adoption_graphs:
    x_sc = bqplot.LinearScale()
    y_sc = bqplot.LinearScale()
    def_tt = bqplot.Tooltip(fields=['name', 'x', 'y'], formats=['', '', '.2f'], labels=['label', 'year', 'value'])
    y_data = []
    x_data = []
    labels = []
    for s in solutions:
        data = s.ua.soln_pds_tot_iunits_reqd()['World']
        y_data.append(data.values)
        x_data = data.index
        labels.append(fullname(s))
    line_chart = bqplot.Lines(x=x_data, y=y_data, scales= {'x': x_sc, 'y': y_sc},
                              tooltip=def_tt, display_legend=True, labels=labels)
    ax_x = bqplot.Axis(scale=x_sc)
    ax_y = bqplot.Axis(scale=y_sc, orientation='vertical')
    x_sc.min = 2020
    x_sc.max = 2050
    fig_iunits = bqplot.Figure(marks=[line_chart], axes=[ax_x, ax_y], legend_location='bottom-left',
                              title='World Adoption - Implementation Units (TW)',
                              legend_text={'font-size': '0.9vw'})
    fig_iunits.layout = {'width': '45%'}

    x_sc = bqplot.LinearScale()
    y_sc = bqplot.LinearScale()
    def_tt = bqplot.Tooltip(fields=['name', 'x', 'y'], formats=['', '', '.2f'], labels=['label', 'year', 'value'])
    y_data = []
    x_data = []
    labels = []
    for s in solutions:
        data = s.tm.ref_tam_per_region()['World']
        y_data.append(data.values)
        labels.append('Total Market-REF:'+fullname(s))

        data = s.tm.pds_tam_per_region()['World']
        y_data.append(data.values)
        labels.append('Total Market-PDS:'+fullname(s))

        data = s.ht.soln_ref_funits_adopted()['World']
        y_data.append(data.values)
        labels.append('Solution-REF:'+fullname(s))

        data = s.ht.soln_pds_funits_adopted()['World']
        y_data.append(data.values)
        x_data = data.index
        labels.append('Solution-PDS:'+fullname(s))
    line_chart = bqplot.Lines(x=x_data, y=y_data, scales= {'x': x_sc, 'y': y_sc},
                              tooltip=def_tt, display_legend=True, labels=labels)
    ax_x = bqplot.Axis(scale=x_sc)
    ax_y = bqplot.Axis(scale=y_sc, orientation='vertical')
    x_sc.min = 2020
    x_sc.max = 2050
    fig_funits = bqplot.Figure(marks=[line_chart], axes=[ax_x, ax_y], legend_location='bottom-left',
                              title='World Growth of Solution - Functional Units (TWh)',
                              legend_text={'font-size': '0.8vw'})
    fig_funits.layout = {'width': '45%'}
    display(ipywidgets.HBox(children=[fig_iunits, fig_funits]))

# --- Financial ---

financial_results_label = blue_label('The Key Financial Results</div>')

financial_text = []
for s in solutions:
    marginal_first_cost = (s.fc.soln_pds_annual_world_first_cost().loc[:2050].sum() - \
        s.fc.soln_ref_annual_world_first_cost().loc[:2050].sum() - \
        s.fc.conv_ref_annual_world_first_cost().loc[:2050].sum()) / 10**9
    net_operating_savings = ((s.oc.conv_ref_cumulative_operating_cost().loc[2050] - \
                             s.oc.conv_ref_cumulative_operating_cost().loc[2020]) - \
                            (s.oc.soln_pds_cumulative_operating_cost().loc[2050] - \
                             s.oc.soln_pds_cumulative_operating_cost().loc[2020])) / 10**9
    lifetime_operating_savings = s.oc.lifetime_cost_forecast()['Marginal Operating Cost Savings'].sum() / 10**9
    financial_text.append([fullname(s), marginal_first_cost, net_operating_savings, lifetime_operating_savings])
financial_text.append(['', 'Billion USD', 'Billion USD', 'Billion USD'])
financial_columns = ['Scenario', 'Marginal First Cost 2020-2050',
                     'Net Operating Savings 2020-2050', 'Lifetime Operating Savings 2020-2050']

financial_heading = ipywidgets.Output()
with financial_heading:
    display(HTML(pd.DataFrame(financial_text, columns=financial_columns).to_html(index=False, float_format=lambda x: '%.2f' % x)))

cost_graphs = ipywidgets.Output()
with cost_graphs:
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(18, 4))
    ax[0].set_title('World Operating Cost Difference')
    ax[0].set_xlim(left=2010, right=2060)
    ax[0].set_xlabel('Billions US$2014')
    matplotlib.ticker.EngFormatter.ENG_PREFIXES[9] = 'B'
    fmty = matplotlib.ticker.EngFormatter(sep='')
    ax[0].yaxis.set_major_formatter(fmty)
    for s in solutions:
        s.oc.soln_pds_annual_operating_cost().loc[2020:2050].plot(ax=ax[0], legend=True, label='Solution Operating Costs/Savings:' + fullname(s))
        s.oc.conv_ref_annual_operating_cost().loc[2020:2050].plot(ax=ax[0], legend=True, label='Conventional Operating Costs/Savings:' + fullname(s))
    plt.show(fig)

# --- Climate ---

climate_results_label = blue_label('The Key Climate Results</div>')

climate_text = []
for s in solutions:
    climate_text.append([fullname(s), (s.c2.co2eq_mmt_reduced().loc[:, 'World'].sum() / 1000)])
climate_text.append(['', 'Gt CO2 (2020-2050)'])
climate_columns = ['Scenario', 'Total Emissions Reduction']

climate_heading = ipywidgets.Output()
with climate_heading:
    display(HTML(pd.DataFrame(climate_text, columns=climate_columns).to_html(index=False, float_format=lambda x: '%.2f' % x)))

emissions_graphs = ipywidgets.Output()
with emissions_graphs:
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(13, 4))
    ax[0].set_title('World Cumulative Emissions Reductions (Gigatons CO2-eq)')
    ax[0].set_xlim(left=2020, right=2050)
    for s in solutions:
        s.c2.co2eq_mmt_reduced()['World'].cumsum().mul(0.001).plot(ax=ax[0], legend=True, label=fullname(s))
    ax[1].set_title('World Cumulative GHG Concentrations Reduction (PPM CO2-eq)')
    ax[1].set_xlim(left=2020, right=2050)
    for s in solutions:
        s.c2.co2eq_ppm_calculator()['CO2-eq PPM'].plot(ax=ax[1], legend=True, label=fullname(s))
    plt.show(fig)

detailed_results = ipywidgets.Output()
with detailed_results:
    display(ipywidgets.VBox([key_results_label, unit_adoption_heading, adoption_heading, adoption_graphs,
                             financial_results_label, financial_heading, cost_graphs,
                             climate_results_label, climate_heading, emissions_graphs]))

# --------------- Variable Meta Analysis --------------

variable_meta_analysis = ipywidgets.Output()
with variable_meta_analysis:
    children = []
    for s in solutions:
        for (name, v) in s.VMAs:
            vma_widget = ipywidgets.Output()
            with vma_widget:
                display(HTML(v.source_data.drop(['Link', 'Source Validation Code', 'License Code'], axis=1).fillna('').to_html()))
            accordion = ipywidgets.Accordion(children=[vma_widget])
            accordion.set_title(0, name)
            vma_widget = ipywidgets.Output()
            with vma_widget:
                display(ipywidgets.HBox(children=[ipywidgets.Label('Summary Placeholder'), accordion]))
            children.append(vma_widget)
    display(ipywidgets.VBox(children=children))

# ------------------ Operating Cost -----------------

children = []
for s in solutions:
    df = pd.concat([s.oc.soln_pds_annual_operating_cost().loc[:2060],
                    s.oc.conv_ref_annual_operating_cost().loc[:2060],
                    s.oc.marginal_annual_operating_cost().loc[:2060]],
                   axis=1)
    oc_table = ipywidgets.Output()
    with oc_table:
        display(HTML(df.to_html()))
    children.append(oc_table)

operating_cost = ipywidgets.Accordion(children=children)
for i, s in enumerate(solutions):
    operating_cost.set_title(i, fullname(s))

# ------------------ First Cost -----------------

children = []
for s in solutions:
    df = pd.concat([s.fc.soln_pds_install_cost_per_iunit().loc[:2060],
                    s.fc.soln_ref_install_cost_per_iunit().loc[:2060],
                    s.fc.conv_ref_install_cost_per_iunit().loc[:2060]],
                   axis=1)
    fc_table = ipywidgets.Output()
    with fc_table:
        display(HTML(df.to_html()))
    children.append(fc_table)

first_cost = ipywidgets.Accordion(children=children)
for i, s in enumerate(solutions):
    first_cost.set_title(i, fullname(s))

In [73]:
solutions = pd.read_csv(os.path.join('data', 'overview', 'solutions.csv'),
                                    index_col=False, skipinitialspace=True, header=0,
                                    skip_blank_lines=True, comment='#')
soln_results = pd.read_csv(os.path.join('data', 'overview', 'soln_results.csv'),
                                    index_col=False, skipinitialspace=True, header=0,
                                    skip_blank_lines=True, comment='#')
solutions = solutions.merge(soln_results, on='Solution', how='left')

colormap = {
    'Materials': 'RebeccaPurple',
    'Electricity Generation': 'Peru',
    'Food': 'FireBrick',
    'Land Use': 'Green',
    'Women and Girls': 'DarkGoldenRod',
    'Transport': 'Teal',
    'Buildings and Cities': 'SteelBlue',
}

def get_color(sector):
    return colormap.get(sector, 'Beige')

soln_data = solutions['CO2eq'].tolist()
soln_labels = solutions['Solution'].tolist()
soln_colors = solutions['Sector'].apply(get_color).tolist()
soln_tt = bqplot.Tooltip(fields=['label', 'size'], formats=['', '0.1f'], labels=['Name', 'GTons-CO2'])
soln_pie = bqplot.Pie(sizes=soln_data, display_labels='none', labels=soln_labels, sort=True, tooltip=soln_tt,
                 display_values=True, values_format='.1f', radius=170, inner_radius=150,
                 colors=soln_colors, stroke='Gainsboro',
                 tooltip_style = {'opacity': 0.7, 'background-color': 'White', 'border': 'none',
                               'border-collapse': 'collapse'})

sectors = solutions.pivot_table(index='Sector',aggfunc=sum)
sctr_data = sectors['CO2eq'].tolist()
sctr_labels = sectors.index.tolist()
sctr_colors = [get_color(v) for v in sctr_labels]
sctr_tt = bqplot.Tooltip(fields=['label', 'size'], formats=['', '0.1f'], labels=['Name', 'GTons-CO2eq'])
sctr_pie = bqplot.Pie(sizes=sctr_data, display_labels='none', labels=sctr_labels, sort=True, tooltip=sctr_tt,
                 display_values=True, values_format='.1f', radius=138, inner_radius=118,
                 colors=sctr_colors, stroke='Gainsboro')
sctr_pie.tooltip_style = {'opacity': 0.7, 'background-color': 'White', 'border': 'none',
                               'border-collapse': 'collapse'}
solution_chart = bqplot.Figure(marks=[soln_pie, sctr_pie], animation_duration=1000)
solution_chart.background_style = {'fill': 'White'}

solution_html = '<table><tr><td><b>Name</b></td><td><b>Sector</b></td><td><b>Model</b></td></tr>'
for row in solutions.itertuples(index=False):
    name = getattr(row, "Solution")
    directory = getattr(row, "DirName")
    sector = getattr(row, "Sector")
    solution_html += '<tr><td>' + name + '</td><td>' + sector + '</td>'
    if directory is not None and directory is not np.nan:
        solution_html += '<td>' + directory + '</td>'
    else:
        solution_html += '<td></td>'
solution_html += '</table>'
solution_list = ipywidgets.Output()
with solution_list:
    display(HTML(solution_html))

overview = ipywidgets.HBox(children=[solution_list, solution_chart])

In [74]:
# ------------------ Create tabs -----------------

tabs = ipywidgets.Tab(children=[overview, detailed_results, variable_meta_analysis, operating_cost, first_cost])
tabs.set_title(0, "Overview")
tabs.set_title(1, "Detailed Results")
tabs.set_title(2, "Variable Meta-Analysis")
tabs.set_title(3, "Operating Cost")
tabs.set_title(4, "First Cost")
display(tabs)